In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow import keras
from keras import losses

2024-10-02 21:53:49.149799: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-02 21:53:49.157115: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 21:53:49.165993: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 21:53:49.168504: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 21:53:49.179396: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


I0000 00:00:1727895230.302430   35265 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727895230.415328   35265 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727895230.415364   35265 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
raw = open('data2.txt', mode='r', encoding='utf-8').readlines()
data = []
for line in raw:
    if line != '\n' and 'Глава' not in line:
        data.append(' '.join(line.split()[1:]))
    
data = [line.replace('\n', ' ').replace('\xa0', ' ') for line in data]
text = ' '.join(data)

In [4]:
text[:100]

'завет ЗАВЕТ Моисея  В начале сотворил Бог небо и землю. Земля же была безвидна и пуста, и тьма над б'

In [5]:
def get_features_target(seq):
    features = seq[:-1]
    target = seq[1:]
    return features, target

BATCH_SIZE = 128

alphabet = np.array(sorted(set(text)))
sym_to_idx = {}
idx_to_sym = {}

for idx, sym in enumerate(alphabet):
    sym_to_idx[sym] = idx
    idx_to_sym[idx] = sym
    
text_idx = np.array([sym_to_idx[char] for char in text])
sequences = Dataset.from_tensor_slices(text_idx).batch(BATCH_SIZE, drop_remainder=True)

dataset = sequences.map(get_features_target)

data = dataset.batch(BATCH_SIZE, drop_remainder=True).repeat()
data = data.prefetch(AUTOTUNE)

I0000 00:00:1727895231.019286   35265 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727895231.019332   35265 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727895231.019344   35265 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727895231.099376   35265 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727895231.099423   35265 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-02

In [6]:
model = keras.Sequential([
    keras.layers.Embedding(len(alphabet), 256),
    keras.layers.LSTM(512, return_sequences=True, stateful=True),
    keras.layers.Dense(len(alphabet))
])

model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(data, epochs=40, verbose=1, steps_per_epoch= len(sequences) // BATCH_SIZE)

Epoch 1/40
  4/243 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.0867 - loss: 4.6230

2024-10-02 21:53:54.217591: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


243/243 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.2436 - loss: 2.8901
Epoch 2/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.4010 - loss: 2.0754
Epoch 3/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.4646 - loss: 1.8323
Epoch 4/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4958 - loss: 1.7033 
Epoch 5/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5168 - loss: 1.6205
Epoch 6/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.5326 - loss: 1.5612
Epoch 7/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5450 - loss: 1.5158
Epoch 8/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5548 - loss: 1.4796
Epoch 9/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5633 - loss: 1.4498
Epoch 10/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5702 - loss: 1.4243
Epoch 11/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5767 - loss: 1.4022
Epoch 12/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/ste

In [7]:
def predict_next(sample, model, tokenizer, vocabulary, n_next, rnd_power, batch_size):
    sample_token = [tokenizer[char] for char in sample]
    predicted = sample_token

    sample_tensor = tf.expand_dims(sample_token, 0)
    sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    
    for _ in range(n_next):
        cur = model(sample_tensor)
        cur = cur[0].numpy() / rnd_power
        cur = tf.random.categorical(cur, num_samples=1)[-1, 0].numpy()
        predicted.append(cur)
        sample_tensor = predicted[-99:]
        sample_tensor = tf.expand_dims([cur], 0)
        sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    res = [vocabulary[i] for i in predicted]
    generated = ''.join(res)
    return generated

In [8]:
print(predict_next(
    sample='б',
    model=model,
    tokenizer=sym_to_idx,
    vocabulary=idx_to_sym,
    n_next=200,
    temperature=0.6,
    batch_size=BATCH_SIZE
))

бом и селениями своими, ибо время согрешающего совершенного скота, о котором сказано: верен Бог их и умер, и освящающих нас, что делается всё, что значит им слово Божие, которое я имею тельцов и не дол


In [9]:
print(predict_next(
    sample='1',
    model=model,
    tokenizer=sym_to_idx,
    vocabulary=idx_to_sym,
    n_next=100,
    temperature=0.2,
    batch_size=BATCH_SIZE
))

1гая в сердцах своих и принятие в слове своём и сказали ему: вот, я возлюбленному своему и служению в
